<a href="https://colab.research.google.com/github/melissatorgbi/LLMCxG_Workshop/blob/main/Live_LLMCxG_Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook uses data from the paper ["A Construction Grammar Corpus of Varying Schematicity: A Dataset for the Evaluation of Abstractions in Language Models"](https://aclanthology.org/2024.lrec-main.22/) to probe LLMs using the metalinguistic task of identifying which sentences belong to specific constructions.

The data consists of 10 distinct English constructions with at least 50 example sentences for each construction. This notebook covers the following:

1.   Loading the Data
2.   Cleaning the Data
3.   Constructing Prompts
4.   Prompting the Model
5.   Evaulation


We use the OpenAI API to prompt a model so an API key is required.

This version of the notebook contains activities to complete as you go through. A complete version of the notebook with solutions will be shared later.

## Imports

In [ ]:
import csv
import pandas as pd
import random
from openai import OpenAI
import re
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


# Set-up

In [ ]:
OPENAI_API_KEY = "replace the text with your API key" #replace the text with your API key

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_response(prompt, model):

  completion = client.chat.completions.create(
      model=model,
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ]
  )

  response = completion.choices[0].message.content
  return response

In [ ]:
def clean_data(df):
  for column in df.columns:
    clean_list = clean_sentences(df[column])
    df[column] = clean_list

  return df

In [ ]:
def get_rows(df, row_num):

  rows = []

  for i in range(row_num):
    row = []
    construction = random.choice(df.columns)
    construction_sentence = df[construction][random.randint(0, df[construction].count()-2)]

    remaining_constructions = df.drop(columns = construction)
    random_construction1 = random.choice(remaining_constructions.columns)
    alternative_sentence1 = df[random_construction1][random.randint(0, df[random_construction1].count()-2)]

    random_construction2 = random.choice(remaining_constructions.columns)
    remaining_constructions = remaining_constructions.drop(columns = random_construction1)
    alternative_sentence2 = df[random_construction2][random.randint(0, df[random_construction2].count()-2)]

    row = [construction_sentence, alternative_sentence1, alternative_sentence2]
    random.shuffle(row)
    row.append(construction)
    row.append(construction_sentence)

    rows.append(row)

  new_df = pd.DataFrame(rows, columns = ['sentence1','sentence2','sentence3','construction','target'])

  return new_df


## Load Data

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/Construction_Grammar_Schematicity_Corpus-CoGS.git

fatal: destination path 'Construction_Grammar_Schematicity_Corpus-CoGS' already exists and is not an empty directory.


In [ ]:
cogs_df = pd.read_csv("Construction_Grammar_Schematicity_Corpus-CoGS/Dataset/CoGs.csv")
cogs_df = cogs_df.loc[:, ~cogs_df.columns.str.match('Unnamed')]
cogs_df = cogs_df.loc[0:49]
cogs_df.head()

,Let Alone,Way Manner,Resultative,Conative,Intransitive Motion,Caused Motion,Causative with CxN,Ditransitive CxN,Comparative Correlative,Much Less
0,"Most wives are too bloody old, let alone mothe...","Tricia backed her way out, never taking her ey...",The man shrieked himself unconscious. (Hoffman),She kicked at the ball. (Hoffman),The fly buzzed into the room. (Hoffman),He sang them out of the room. (Hoffman),She loaded the truck with books. (Hoffman),Jack passed her the salt. (Hoffman),"The harder they come, the harder they fall. (AMR)",When my dad catches swarms sometimes he doesn'...
1,"A ceasefire, let alone lasting peace, will tak...",The taxi nosed its way back into the traffic a...,Firefighters cut the man free. (Hoffman),He clutched at the branch. (Hoffman),He ran out of the house. (Hoffman),She wiggled her feet out of the boots. (Hoffman),He sprayed the walls with paint. (Hoffman),The waiter served them their dinner. (Hoffman),"The longer he is around, the more miserable I ...","Not that many of us can fly off to Guatemala, ..."
2,It is difficult enough for an individual to be...,"As she felt her way forward, suddenly a knight...",He had often drank himself silly. (Hoffman),They shot at the sheriff. (Hoffman),People strolled along the river. (Hoffman),They laughed the actor off the stage. (Hoffman),They heaped the plate with mashed potatoes. (H...,She sent him an email. (Hoffman),"The more I studied, the less I understood. (AMR)","I never thought I'd meet him, much less in Aus..."
3,I would be distressed to hear of any ladies re...,"If this were to happen, would it not be unfair...","It JERKS you awake with the first sentence , h...","With her free hand, she tugged at the strap of...",It shouldn't be - you should be able to DANCE ...,I THREW the stone across the river. (AMR),Kevin lifted the trumpet and filled the room w...,The above kind of exchange AFFORDS students th...,The more the merrier. (AMR),Those were not my first (much less only) thoug...
4,"None of these arguments is notably strong, let...",Through the French windows and across the lawn...,These agencies RENDER themselves ineffective b...,She tugged at the glass door. (COCA),"When the IED detonates, this copper cup turns ...",The RN can still launch a task force that woul...,Dude has filled the room with flowers and tedd...,She baked her sister a cake. (Goldberg),"It seems the older the patient, the less effec...",You can not hear (much less understand) anythi...


In [ ]:
# The data contains 10 constructions as the column names
cogs_df.columns

Index(['Let Alone', 'Way Manner', 'Resultative', 'Conative',
       'Intransitive Motion', 'Caused Motion', 'Causative with CxN',
       'Ditransitive CxN', 'Comparative Correlative ', 'Much Less '],
      dtype='object')

In [ ]:
# You can explore specific sentences by changing the construction name and the index
cogs_df["Caused Motion"][0]

'He sang them out of the room.  (Hoffman)'

## Task 1

**Task 1: Identifying Constructions**

Can you describe each of the constructions by exploring the examples?

## Task 2

**Task 2: Data Cleaning**

Can you identify elements of the input that would require modification?

# Clean Data

Before we use the data in our prompts we want to make sure it is in the format that we want.

In [ ]:
# before cleaning
print("Example sentences before cleaning:\n\n"+cogs_df["Ditransitive CxN"][3])

Example sentences before cleaning:

The above kind of exchange AFFORDS students the opportunity to ask such questions in a legitimate , dialogical environment . (COCA)


## Task 3: Data cleaning
Complete the function below to clean the input sentences and return the clean sentences.

In [ ]:
# Add your code to clean the input sentences here.
def clean_sentences(sentences) :
    return sentences

In [ ]:
# you could try experimenting with some of the following python methods

test_string = "The above kind of exchange AFFORDS students the opportunity to ask such questions in a legitimate , dialogical environment . (COCA)"
test_string = test_string.lower()
test_string = test_string.replace(" .",".")
test_string = test_string.replace(" ,",",")
test_string = test_string.replace("(coca)", "")
test_string = test_string.capitalize()
print(test_string)

The above kind of exchange affords students the opportunity to ask such questions in a legitimate, dialogical environment. 


In [ ]:
# code to clean the input sentences.
def clean_sentences(sentences) :
    clean_sentences = list()
    for sentence in sentences :
        sentence = re.sub( r"\s*\([^\)]*?\)\s*$", r"", sentence )
        sentence = re.sub( r"\s+.$", '.', sentence )
        sentence = re.sub( r"\s,", ",", sentence )
        if sentence != '' :
            new_sentence = list()
            for word, pos in nltk.pos_tag( sentence.split( ' ' ) ) :
                verbs   = [ (i.pos()=='v') for i in wordnet.synsets(word) ]
                wn_verb = verbs.count( True )  > verbs.count( False )
                if word.isupper() and ( 'VB' in pos or wn_verb ) :
                    word = word.lower()
                new_sentence.append( word )
            sentence = ' '.join( new_sentence )
            clean_sentences.append( sentence )
    return clean_sentences

In [ ]:
# This cleans the data using the clean_sentences function you have written and
# the clean_data function found in the set-up section of this notebook

cogs_df = clean_data(cogs_df)

In [ ]:
# after cleaning
print("Example sentences after cleaning:\n\n"+cogs_df["Ditransitive CxN"][3])

Example sentences after cleaning:

The above kind of exchange affords students the opportunity to ask such questions in a legitimate, dialogical environment.


# Constructing Prompts

The task involves giving the model 3 sentences from different constructions. We want to the model to choose one sentence out of three sentences provided that matches a specific construction.

We are essentially asking the model which one of the sentences are instances of the specific construction?

In the code blocks below we manually extract relevant parts of our data to show an example of what you may consider putting in the prompt.

In task 4 below you will construct a prompt base. The prompt_base is used for prompt creation later in this notebook when we automate the process the prompt the model multiple times.

In [ ]:
# For demonstratuin purposes we manually extract the first example of the let alone construction in the data,
# along with 2 other sentences from random constructions

construction = "Caused Motion"
construction_sentence = cogs_df[construction][0]
remaining_constructions = cogs_df.drop(columns = construction)
random_construction1 = random.choice(remaining_constructions.columns)
alternative_sentence1 = cogs_df[random_construction1][0]
random_construction2 = random.choice(remaining_constructions.columns)
remaining_constructions = remaining_constructions.drop(columns = random_construction1)
alternative_sentence2 = cogs_df[random_construction2][0]

print("Target Construction: {}\nSentence: {}\n".format(construction, construction_sentence))
print("Alternative Construction: {}\nSentence: {}\n".format(random_construction1, alternative_sentence1))
print("Alternative Construction: {}\nSentence: {}\n".format(random_construction2, alternative_sentence2))

Target Construction: Caused Motion
Sentence: He sang them out of the room.

Alternative Construction: Way Manner
Sentence: Tricia backed her way out, never taking her eyes off me or letting her bearings slip.

Alternative Construction: Comparative Correlative 
Sentence: The harder they come, the harder they fall.



## Task 4: The Prompt

**Task 4: The Prompt**

What question do you want to ask the model for this task? Add text and/or variables to the prompt_base string below.

For the final prompt_base please use the python string format() method and variables created in the code block above as follows: prompt_base.format(construction, construction_sentence, alternative_sentence1, alternative_sentence2)

If you are unformiliar with the python string format() method, you can look at the [python docs](https://docs.python.org/3/tutorial/inputoutput.html#the-string-format-method) or [this tutorial](https://www.geeksforgeeks.org/python-string-format-method/).

An example of using the string format() method is shown in the code block below.

In [ ]:
# example of using the string format() method

format_example_base = "The construction is {}"
format_example = "The construction is {}".format(construction)
print(format_example)

The construction is Caused Motion


In [ ]:
# edit the string below to form your prompt base

prompt_base = """Which of the following setnence is an instance of the {} construction:
{}
{}
{}
Answer:
"""

In [ ]:
# Example of what your prompt given to the model would look like

prompt = prompt_base.format(construction, construction_sentence, alternative_sentence1, alternative_sentence2)
print(prompt)

Which of the following setnence is an instance of the Caused Motion construction:
He sang them out of the room.
Tricia backed her way out, never taking her eyes off me or letting her bearings slip.
The harder they come, the harder they fall.
Answer:



# Prompting the Model

Using the example prompt we create above, we will prompt one of the OpenAI models and get the models reponse.

In [ ]:
print( prompt )
response = get_response(prompt, "gpt-4o-mini")

Which of the following setnence is an instance of the Caused Motion construction:
He sang them out of the room.
Tricia backed her way out, never taking her eyes off me or letting her bearings slip.
The harder they come, the harder they fall.
Answer:



In [ ]:
print(response)

The sentence that is an instance of the Caused Motion construction is:

"He sang them out of the room."

In this sentence, the use of "sang" indicates that the action causes the movement of "them" out of the room, which fits the characteristics of the Caused Motion construction.


# Evaluation

To evaluate the model's performance we compare the model's output to the the correct answer.

We calculate exact accuracy and relaxed accuracy. Exact accuracy is when the models response perfectly matches the correct answer. Relaxed accuracy does not require a perfect match but the model's reponse should appear in the correct answer or the correct answer should appear in the model's response.

In [ ]:
print("model's response:\n"+response)
print("\ncorrect answer:\n"+construction_sentence)

model's response:
The sentence that is an instance of the Caused Motion construction is:

"He sang them out of the room."

In this sentence, the use of "sang" indicates that the action causes the movement of "them" out of the room, which fits the characteristics of the Caused Motion construction.

correct answer:
He sang them out of the room.


## Task 5: Evaluation
How would you eveluate the output?

In [ ]:
# add to the fuction below to check if the models response matches the target sentence
def calculate_exact_score( target, response ) :
  return response == target


# do we need any other method of calculating the score? if not you can skip the next function.
def calculate_alternative_score( target, response ):
  return target in response

In [ ]:
calculate_exact_score( construction_sentence, response)

False

# Prompting with Multiple Examples

We have gone through the prompt construction, prompting the model and evaluating the model's response using one example. Now we will do the same thing multiple times for the same task.

To make this easier we will first transform the data so that each row has everything we need to create an individual prompt and evalution. Each row will contain the following:


1.   A sentence from the target construction
2.   A sentence from a random construction
3.   A sentence from another random construction

These first 3 sentence are randomly shuffled so may appear in any order (labeled as "sentence1", "sentence2", "sentence3")

4.   The name of the target construction (labeled as "construction")
5.   The correct/target sentence (labeled as "target")

We then prompt the model, getting responses for all rows in the data and then evaluate the results.

In [ ]:
# The get_rows function that transforms the data can be found in the set up section of this notebook
cogsn_df = get_rows(cogs_df, 10)

In [ ]:
cogsn_df.head(10)

,sentence1,sentence2,sentence3,construction,target
0,The above kind of exchange affords students th...,You can not hear (much less understand) anythi...,This nice man probably just wanted Mother to K...,Ditransitive CxN,The above kind of exchange affords students th...
1,"And if they get generators, where are they goi...",Alan's finger stabbed at the button on the fla...,"Ever notice that the more gridlock there is, t...",Much Less,"And if they get generators, where are they goi..."
2,Freshly ground coffee beans filled the room wi...,The practical man thought his way through thes...,"The longer the rain lasted, the more quickly t...",Way Manner,The practical man thought his way through thes...
3,"And if they get generators, where are they goi...","They sent him a bill for $4,000",The idea of everyone writing a letter for thei...,Causative with CxN,The idea of everyone writing a letter for thei...
4,The alarm clock ticked the baby awake.,Thousands of mine workers and supporters plan ...,"She was a big, bosomy woman who couldn't work ...",Resultative,The alarm clock ticked the baby awake.
5,"James Jackson and Company of Augusta, the seco...",We picked out all of the best songs from the t...,He ate himself sick.,Resultative,He ate himself sick.
6,People strolled along the river.,"Chinese officials stated on May 14, 2007 that ...","The neocolonial relationship remains, as was m...",Intransitive Motion,People strolled along the river.
7,My mom bought me these skates.,Mr Brown claimed that the Government had faile...,The fewer things we make the more sustainable ...,Comparative Correlative,The fewer things we make the more sustainable ...
8,Saudi flooded the market with cheap oil then s...,They coughed their way through All Things Brig...,"China confirmed on January 23, 2007 after 12 d...",Way Manner,They coughed their way through All Things Brig...
9,The tornado that advanced through southern Okl...,they can't even recognize a threat much less a...,"The more we get together, the happier we'll be",Comparative Correlative,"The more we get together, the happier we'll be"


In [ ]:
# This code loops through the data
# uses the base prompt to generate a prompt for each row of the data
# prompts the openai model and save the reponse

exact_scores = []
alternative_scores = []
responses = []

for index in range(len(cogsn_df)):

  # creating prompt
  prompt = prompt_base.format(
      cogsn_df.construction.iloc[index],
      cogsn_df.sentence1.iloc[index],
      cogsn_df.sentence2.iloc[index],
      cogsn_df.sentence3.iloc[index])

  # getting reponse from the model
  response = get_response(prompt, "gpt-4o-mini")

  # evaluation
  target = cogsn_df.target.iloc[index]
  exact_score = calculate_exact_score( target, response )
  relaxed_score = calculate_alternative_score( target, response )

  # adding response and evaluation to data
  responses.append(response)
  exact_scores.append(exact_score)
  alternative_scores.append(relaxed_score)

# adding response and evaluation to existing data frame (df)
cogsn_df["response"] = responses
cogsn_df["exact score"] = exact_scores
cogsn_df["alternative score"] = alternative_scores

In [ ]:
# This shows the updated data with additional columns containing the model's response,
# exact match and relaxed match.

cogsn_df.head(10)

,sentence1,sentence2,sentence3,construction,target,response,exact score,alternative score
0,The above kind of exchange affords students th...,You can not hear (much less understand) anythi...,This nice man probably just wanted Mother to K...,Ditransitive CxN,The above kind of exchange affords students th...,The sentence that is an instance of the Ditran...,False,False
1,"And if they get generators, where are they goi...",Alan's finger stabbed at the button on the fla...,"Ever notice that the more gridlock there is, t...",Much Less,"And if they get generators, where are they goi...","The instance of the ""Much Less"" construction i...",False,True
2,Freshly ground coffee beans filled the room wi...,The practical man thought his way through thes...,"The longer the rain lasted, the more quickly t...",Way Manner,The practical man thought his way through thes...,The sentence that is an instance of the Way Ma...,False,True
3,"And if they get generators, where are they goi...","They sent him a bill for $4,000",The idea of everyone writing a letter for thei...,Causative with CxN,The idea of everyone writing a letter for thei...,"The sentence ""They sent him a bill for $4,000""...",False,False
4,The alarm clock ticked the baby awake.,Thousands of mine workers and supporters plan ...,"She was a big, bosomy woman who couldn't work ...",Resultative,The alarm clock ticked the baby awake.,The sentence that is an instance of the Result...,False,True
5,"James Jackson and Company of Augusta, the seco...",We picked out all of the best songs from the t...,He ate himself sick.,Resultative,He ate himself sick.,The sentence that is an instance of the Result...,False,True
6,People strolled along the river.,"Chinese officials stated on May 14, 2007 that ...","The neocolonial relationship remains, as was m...",Intransitive Motion,People strolled along the river.,The sentence that is an instance of the Intran...,False,True
7,My mom bought me these skates.,Mr Brown claimed that the Government had faile...,The fewer things we make the more sustainable ...,Comparative Correlative,The fewer things we make the more sustainable ...,The sentence that is an instance of the Compar...,False,True
8,Saudi flooded the market with cheap oil then s...,They coughed their way through All Things Brig...,"China confirmed on January 23, 2007 after 12 d...",Way Manner,They coughed their way through All Things Brig...,"The ""Way Manner"" construction refers to a part...",False,False
9,The tornado that advanced through southern Okl...,they can't even recognize a threat much less a...,"The more we get together, the happier we'll be",Comparative Correlative,"The more we get together, the happier we'll be",The sentence that is an instance of the Compar...,False,True


In [ ]:
print( cogsn_df.iloc[0] )

sentence1            The above kind of exchange affords students th...
sentence2            You can not hear (much less understand) anythi...
sentence3            This nice man probably just wanted Mother to K...
construction                                          Ditransitive CxN
target               The above kind of exchange affords students th...
response             The sentence that is an instance of the Ditran...
exact score                                                      False
alternative score                                                False
Name: 0, dtype: object


## Task 7 Calculate accuracy

In [ ]:
# how would you calculate accuracy? Add code to this code block

print( sum( cogsn_df["exact score"] ) / len( cogsn_df["exact score"]) )

0.0


In [ ]:
print( sum( cogsn_df["alternative score"] ) / len( cogsn_df["alternative score"]) )

0.7
